In [41]:
import matplotlib.pyplot as plt
import numpy as np
import astropy.units as u
from datetime import datetime
import pickle, json, sys, os, glob
import pandas as pd
from scipy.optimize import curve_fit
from scipy.stats import chi2
from scipy import optimize
import subprocess

from astropy.coordinates     import SkyCoord
from lstchain.io.config      import get_standard_config
from ctapipe.io              import read_table
import tables

# Other auxiliar scripts
sys.path.insert(0, os.getcwd() + "/../scripts/")
import auxiliar as aux
import geometry as geom
import lstpipeline

import logging
logger = logging.getLogger(__name__)
logger.addHandler(logging.StreamHandler())
logger.setLevel(logging.INFO)

In [42]:
""" Source name in order to just complete the results file, and
in order to improve run organization."""
source_name = "crab"

# Standard paths for data in the IT cluster ---------
root_dl1 = "/fefs/aswg/data/real/DL1/*/v0.*/tailcut84/"
root_rfs = "/fefs/aswg/data/models/AllSky/20230901_v0.10.4_allsky_base_prod/"
root_mcs = "/fefs/aswg/data/mc/DL2/AllSky/20230901_v0.10.4_allsky_base_prod/TestingDataset/"

# Root path of this script
root = os.getcwd() + "/"
# Path to store the configuration file we are going to use
config_file = root + "config/standard_config.json"
# Path to store objects
root_objects = root + f"objects/"
# Data main directory
root_data = root + f"../../data/cherenkov_transparency_corrections/{source_name}/"


# Directories for the data
dir_dl1b_scaled = root_data + "dl1_scaled/"
dir_dl1m_scaled = root_data + "dl1_merged_scaled/"
dir_dl2_scaled  = root_data + "dl2_scaled/"
dir_dl2         = root_data + "dl2/"
dir_dl3_scaled_base = root_data + "dl3_scaled/"
dir_dl3_base    = root_data + "dl3/"
dir_irfs        = root_data + "irfs/"

In [38]:
dl3_runs_individual_scaled = glob.glob(dir_dl3_scaled_base + "*/dl3_LST-1.Run?????.fits")
dl3_runs_individual        = glob.glob(dir_dl3_base + "*/dl3_LST-1.Run?????.fits")

In [39]:
for dl3_runs, dir_move in zip([dl3_runs_individual_scaled, dl3_runs_individual], [dir_dl3_scaled_base, dir_dl3_base]):
    command = f"cp"
    
    for file in dl3_runs:
        command = command + f" {file}"
    
    # moving to main folder
    command = command + " " + dir_move
    
    subprocess.run(command, shell=True)

### Add DL3 index files

In [40]:
command = f"lstchain_create_dl3_index_files --input-dl3-dir {dir_dl3_base} --overwrite"
logger.info(command)
subprocess.run(command, shell=True)

command = f"lstchain_create_dl3_index_files --input-dl3-dir {dir_dl3_scaled_base} --overwrite"
logger.info(command)
subprocess.run(command, shell=True)

logger.info(f"\nFinished with the dl3 process")

All dl3 files created 100%


Creating index files...
All dl3 files created 100%


Creating index files...
All dl3 files created 100%


Creating index files...
lstchain_create_dl3_index_files --input-dl3-dir /fefs/aswg/workspace/juan.jimenez/cherenkov_transparency_corrections/data_processing_srunwise/../../data/cherenkov_transparency_corrections/crab/dl3/ --overwrite
lstchain_create_dl3_index_files --input-dl3-dir /fefs/aswg/workspace/juan.jimenez/cherenkov_transparency_corrections/data_processing_srunwise/../../data/cherenkov_transparency_corrections/crab/dl3/ --overwrite
lstchain_create_dl3_index_files --input-dl3-dir /fefs/aswg/workspace/juan.jimenez/cherenkov_transparency_corrections/data_processing_srunwise/../../data/cherenkov_transparency_corrections/crab/dl3/ --overwrite
2024-02-27 13:31:31,527 ERROR [lstchain.high_level.hdu_table] (hdu_table.create_hdu_index_hdu): Run 2914 does not contain HDU BACKGROUND
2024-02-27 13:31:31,528 ERROR [lstchain.high_level.hdu_table] (hdu_table.c